In [2]:
import cv2
import pytesseract
import numpy as np

In [ ]:
# Set Tesseract path (adjust if Tesseract is installed in a custom location)
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Load images
images = [
    cv2.imread("path_to_image_with_russian_plate_close.jpg"),
    cv2.imread("path_to_image_with_russian_plate_far.jpg"),
    cv2.imread("path_to_image_with_non_russian_plate.jpg")
]

# Load the trained Haar Cascade for Russian license plates
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_russian_plate_number.xml")

# Define a function to preprocess and detect plates
def detect_license_plate(image, cascade):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Adjustments to improve detection in different lighting
    gray = cv2.equalizeHist(gray)
    plates = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 40))
    
    for (x, y, w, h) in plates:
        # Draw red bounding box
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
        # Return cropped plate region
        return image, gray[y:y+h, x:x+w]
    return image, None

# Detect plates and show results
detected_images = []
cropped_plates = []
for img in images:
    result_img, plate_region = detect_license_plate(img, plate_cascade)
    detected_images.append(result_img)
    cropped_plates.append(plate_region)

# Display results with bounding boxes
for i, detected_img in enumerate(detected_images):
    cv2.imshow(f"Detected License Plate Image {i+1}", detected_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
def preprocess_for_ocr(plate_img):
    # Resize to standard dimensions
    plate_img = cv2.resize(plate_img, (300, 75))
    
    # Thresholding for better character segmentation
    _, thresh = cv2.threshold(plate_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return thresh

# Function to recognize characters in each cropped plate region
def recognize_characters(plate_img):
    if plate_img is None:
        return "Plate Not Detected"
    # Preprocess plate for OCR
    processed_plate = preprocess_for_ocr(plate_img)
    
    # Use Tesseract OCR to read the text
    text = pytesseract.image_to_string(processed_plate, config='--psm 8')
    return text.strip()

# Recognize characters in each detected license plate
for i, plate in enumerate(cropped_plates):
    plate_text = recognize_characters(plate)
    print(f"License Plate Text for Image {i+1}: {plate_text}")

    # Display the processed plate image for visual inspection
    if plate is not None:
        cv2.imshow(f"Processed Plate {i+1}", preprocess_for_ocr(plate))

cv2.waitKey(0)
cv2.destroyAllWindows()
